In [1]:
from datasets import load_dataset

dataset = load_dataset("iluvvatar/RuNNE", trust_remote_code=True)

In [2]:
train = dataset['train'].to_pandas().set_index('id')
train

,text,entities
id,,
0,Ким Чен Нама убили с помощью запрещённого химо...,"[0 12 PERSON, 56 72 ORGANIZATION, 64 72 COUNTR..."
1,Смена портретов на долларах\nГарриет Табмен\nК...,"[19 27 MONEY, 28 42 PERSON, 52 67 MONEY, 68 77..."
2,Новым генсеком ООН станет португалец Гутерреш\...,"[6 14 PROFESSION, 15 18 ORGANIZATION, 26 36 NA..."
3,Вахту принял\n\nУоррен Баффет назвал своего по...,"[14 27 PERSON, 199 209 DATE, 232 260 PROFESSIO..."
4,В Японии скончался старейший житель Земли — Дз...,"[2 8 COUNTRY, 79 96 DATE, 116 122 COUNTRY, 134..."
...,...,...
836,Электрик руководит социал-демократами Дюссельд...,"[0 8 PROFESSION, 68 72 ORGANIZATION, 73 101 TI..."
839,Мужчина женился на тёще\n\nРумынский мужчина у...,"[25 34 NATIONALITY, 129 138 PERSON, 154 161 CI..."
843,Названы лауреаты премии «World Press Photo»\nФ...,"[61 67 COUNTRY, 68 78 PERSON, 258 286 ORGANIZA..."


In [48]:
train.iloc[0]['text']

'Ким Чен Нама убили с помощью запрещённого химоружия\n\nVX\nПолиция Малайзии установила вещество, с помощью которого был убит Ким Чен Нам — брат лидера КНДР Ким Чен Ына, — это отравляющее вещество нервно-паралитического действия (VX Nerve Agent), которое является химическим оружием.\n\nВ пресс-релизе малайзийской полиции говорится:\n\n\nЦентр анализа химического оружия департамента полиции провёл анализы проб, взятых со слизистой оболочки глаза и лица умершего.\n\nВ нём поясняется, что VX причислен к химоружию «Конвенцией о запрещении разработки, производства, накопления и применения химического оружия и о его уничтожении» от 1997 года.\n\nПозднее выяснилось, что у одной из подозреваемых в убийстве Ким Чен Нама проявлялись симптомы отравления этим веществом.\n\nКим Чен Нам, старший брат лидера КНДР Ким Чен Ына, скончался 13 февраля 2017 года в Малайзии. По предварительным данным, его отравили в аэропорту, когда он направлялся из Малайзии в Макао. По подозрению в причастности к убийству

In [4]:
train.iloc[0]['entities']

array(['0 12 PERSON', '56 72 ORGANIZATION', '64 72 COUNTRY',
       '122 133 PERSON', '141 152 PROFESSION', '148 152 COUNTRY',
       '153 164 PERSON', '296 308 COUNTRY', '330 362 ORGANIZATION',
       '363 383 ORGANIZATION', '620 632 DATE', '695 707 PERSON',
       '757 768 PERSON', '783 794 PROFESSION', '790 794 COUNTRY',
       '795 806 PERSON', '818 838 DATE', '841 849 COUNTRY',
       '928 936 COUNTRY', '939 944 STATE_OR_PROVINCE', '986 998 PERSON',
       '1009 1012 NUMBER', '1033 1041 COUNTRY', '1044 1053 COUNTRY',
       '1055 1071 ORGANIZATION', '1063 1071 COUNTRY',
       '296 316 ORGANIZATION', '506 618 LAW', '13 18 EVENT',
       '882 890 EVENT', '977 985 EVENT', '1119 1127 EVENT',
       '117 121 EVENT', '686 694 EVENT', '330 383 ORGANIZATION',
       '56 63 ORGANIZATION', '309 316 ORGANIZATION', '808 817 EVENT',
       '1055 1062 ORGANIZATION'], dtype=object)

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
inputs = tokenizer(dataset["train"][0]['text'], is_split_into_words=False)
inputs.tokens()[:10]

['[CLS]',
 'Ким',
 'Чен',
 'Нам',
 '##а',
 'убили',
 'с',
 'помощью',
 'запрещён',
 '##ного']

In [1]:
inputs.word_ids()[:10]

NameError: name 'inputs' is not defined

In [16]:
29*28*27*26*25*24 / 1e6

342.0144

In [22]:
from tqdm import tqdm
from collections import defaultdict

diffs = defaultdict(int)

for _, row in tqdm(train.iterrows(), total=train.shape[0]):
    for ner_sample in row['entities']:
        bos_idx, eos_idx, ner_class = ner_sample.split()
        bos_idx, eos_idx = int(bos_idx), int(eos_idx)
        diffs[eos_idx - bos_idx] += 1

100%|█████████████████████████████████████████████████████████████████████████████| 461/461 [00:00<00:00, 10973.00it/s]


In [26]:
# the simpliest approach, train one classifier text->class and input all possible texts in a range?
total_samples = sum(diffs.values())

accumulative = 0
for key, val in sorted(list(diffs.items()), key=lambda p: -p[0]):
    accumulative += val * 100 / total_samples
    print(key, round(val * 100 / total_samples, 3), accumulative)

255 0.004 0.0036045128500883108
231 0.004 0.0072090257001766215
230 0.004 0.010813538550264932
221 0.004 0.014418051400353243
220 0.004 0.018022564250441554
215 0.004 0.021627077100529864
206 0.004 0.025231589950618173
196 0.004 0.028836102800706483
194 0.004 0.03244061565079479
181 0.004 0.0360451285008831
169 0.004 0.03964964135097141
158 0.004 0.04325415420105972
142 0.004 0.04685866705114803
130 0.004 0.05046317990123634
125 0.004 0.05406769275132465
123 0.004 0.05767220560141296
122 0.004 0.06127671845150127
120 0.007 0.0684857441516779
118 0.004 0.0720902570017662
117 0.004 0.07569476985185451
116 0.004 0.07929928270194282
115 0.004 0.08290379555203113
112 0.007 0.09011282125220775
111 0.004 0.09371733410229606
110 0.004 0.09732184695238437
109 0.007 0.10453087265256099
108 0.004 0.1081353855026493
104 0.004 0.11173989835273761
103 0.004 0.11534441120282592
101 0.004 0.11894892405291423
100 0.011 0.12976246260317917
99 0.007 0.1369714883033558
98 0.007 0.1441805140035324
97 0.007

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("yqelz/xml-roberta-large-ner-russian")
model = AutoModelForTokenClassification.from_pretrained("yqelz/xml-roberta-large-ner-russian")

tokenizer_config.json:   0%|          | 0.00/421 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [48]:
import torch

inputs = tokenizer('Ким Чен Нама убили с помощью запрещённого химоружия\n\nПолиция Малайзии установила вещество', return_tensors="pt")

In [49]:
logits = model(**inputs).logits

In [51]:
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

''

In [52]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="yqelz/xml-roberta-large-ner-russian")

In [70]:
res = pipe(train.iloc[0]['text'])
res

[{'entity': 'B-PER',
  'score': 0.8024856,
  'index': 1,
  'word': '▁Ким',
  'start': 0,
  'end': 3},
 {'entity': 'I-PER',
  'score': 0.73280287,
  'index': 2,
  'word': '▁Че',
  'start': 4,
  'end': 6},
 {'entity': 'I-PER',
  'score': 0.83399314,
  'index': 3,
  'word': 'н',
  'start': 6,
  'end': 7},
 {'entity': 'I-PER',
  'score': 0.7271455,
  'index': 4,
  'word': '▁На',
  'start': 8,
  'end': 10},
 {'entity': 'I-PER',
  'score': 0.5694821,
  'index': 5,
  'word': 'ма',
  'start': 10,
  'end': 12},
 {'entity': 'B-LOC',
  'score': 0.9966641,
  'index': 23,
  'word': '▁Малайз',
  'start': 64,
  'end': 70},
 {'entity': 'I-LOC',
  'score': 0.9948118,
  'index': 24,
  'word': 'ии',
  'start': 70,
  'end': 72},
 {'entity': 'B-PER',
  'score': 0.88216025,
  'index': 35,
  'word': '▁Ким',
  'start': 122,
  'end': 125},
 {'entity': 'I-PER',
  'score': 0.79826176,
  'index': 36,
  'word': '▁Че',
  'start': 126,
  'end': 128},
 {'entity': 'I-PER',
  'score': 0.8477307,
  'index': 37,
  'word'

In [71]:
def simplify_res(output: list[dict]) -> list[str]:
    output = output.copy()
    output.append({
        'entity': 'B-END',
        'index': int(1e9),
        'word': '',
        'start': int(1e9),
        'end': int(1e9)
    })
    
    start = output[0]['start']
    end = output[0]['end']
    word = output[0]['word']
    
    ans = []
    for i in range(1, len(output)):
        if output[i]['entity'].startswith('I'):
            end = output[i]['end']
            word += output[i]['word']
        else:
            ans.append(f"{start} {end} {output[i-1]['entity']} {word}")
            
            start = output[i]['start']
            end = output[i]['end']
            word = output[i]['word']
    
    return ans
            
            
            

In [72]:
simplify_res(res)

['0 12 I-PER ▁Ким▁Чен▁Нама',
 '64 72 I-LOC ▁Малайзии',
 '122 152 I-ORG ▁Ким▁Чен▁НамНДР',
 '153 224 I-ORG ▁Ким▁Чен▁Ынано-паралитического▁действия',
 '296 304 I-LOC ▁малайзий',
 '330 362 I-ORG ▁Центр▁анализа▁химического▁оружия',
 '363 383 I-ORG ▁департамента▁полиции',
 '505 632 I-ORG ▁«Конвенцией▁о▁запрещении▁разработки,▁производства,▁накопления▁и▁применения▁химического▁оружия▁и▁о▁его▁уничтожении»▁от▁1997▁года',
 '695 705 I-PER ▁Ким▁Чен▁На',
 '757 768 I-PER ▁Ким▁Чен▁Нам',
 '790 794 I-ORG ▁КНДР',
 '795 806 I-PER ▁Ким▁Чен▁Ына',
 '841 849 I-LOC ▁Малайзии',
 '928 936 I-LOC ▁Малайзии',
 '939 944 I-LOC ▁Макао',
 '1033 1041 I-LOC ▁Вьетнама',
 '1044 1053 I-LOC ▁Индонезии',
 '1063 1071 I-LOC ▁Малайзии']

In [73]:
train.iloc[0]['entities']

array(['0 12 PERSON', '56 72 ORGANIZATION', '64 72 COUNTRY',
       '122 133 PERSON', '141 152 PROFESSION', '148 152 COUNTRY',
       '153 164 PERSON', '296 308 COUNTRY', '330 362 ORGANIZATION',
       '363 383 ORGANIZATION', '620 632 DATE', '695 707 PERSON',
       '757 768 PERSON', '783 794 PROFESSION', '790 794 COUNTRY',
       '795 806 PERSON', '818 838 DATE', '841 849 COUNTRY',
       '928 936 COUNTRY', '939 944 STATE_OR_PROVINCE', '986 998 PERSON',
       '1009 1012 NUMBER', '1033 1041 COUNTRY', '1044 1053 COUNTRY',
       '1055 1071 ORGANIZATION', '1063 1071 COUNTRY',
       '296 316 ORGANIZATION', '506 618 LAW', '13 18 EVENT',
       '882 890 EVENT', '977 985 EVENT', '1119 1127 EVENT',
       '117 121 EVENT', '686 694 EVENT', '330 383 ORGANIZATION',
       '56 63 ORGANIZATION', '309 316 ORGANIZATION', '808 817 EVENT',
       '1055 1062 ORGANIZATION'], dtype=object)

In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
input_idxs = tokenizer(['я люблю когда голые волосатые мужики сосут', 'котята это вау'], 
                       return_tensors="pt", padding=True, truncation=True, max_length=256)
input_idxs
output = model(**input_idxs)['last_hidden_state']
output.shape

torch.Size([2, 12, 768])

In [45]:
output.shape

torch.Size([2, 12, 768])

In [39]:
input_idxs['input_ids']

tensor([[  101,   877, 49775,  5318, 48581, 21595, 24129,  5891,  3437,   102],
        [  101, 16022, 21295,  3998, 12962,   876,   102,     0,     0,     0]])